# Imports

In [1]:
import pandas as pd

# Defining a handy function

In [2]:
def TransformColumns(df):
    """Replaces the spaces with underlines and converts to lowecase the column names of a given DataFrame `df`.
    
    Parameters
    ----------
    df : DataFrame with messy column names
    
    Returns
    -------
    None
    
    Raises
    ------
    AssertionError
        * If `df` is not a ``pandas.DataFrame``.
    """
    
    try:
        assert type(df) is pd.core.frame.DataFrame, "Invalid input. Must be a DataFrame."
        df.columns = [c.lower().replace(' ', '_') for c in df.columns]
        return None
    except AssertionError as msg:
        print(msg)
        return None

# Implementing solution

### Declaring variables

In [3]:
dictMonths = {1: 'jan', 2:'fev', 3:'mar', 4:'abr', 5:'mai', 6:'jun', 7:'jul', 8:'ago', 9:'set', 10:'out', 11:'nov', 12:'dez'}

varMonth = input("Enter the first MONTH of the forecast that should be reviewed: ") #8
varYear = input("Enter the YEAR of the first month that should be reviewed: ") #2019

varQuery = 'ano == '+str(varYear)+' or (ano == '+str(int(varYear) - 1)+' and mes >= '+str(varMonth)+')'

Enter the first MONTH of the forecast that should be reviewed: 8
Enter the YEAR of the first month that should be reviewed: 2019


### Importing and preparing DataFrames

Here all DataFrames used in this solution are imported and transformed with the previously declared function TransformColumns. They are namely: <br>
- dfGroupAnalyst: A list of analysts to which each product group is assigned;
- dfProd: A product log, with basic infos about all the registered products;
- dfStockECMandDC: An inventory of all product quantities destined to e-commerce and physical stores;
- dfStockStores: An inventory of all product quantities on the physical stores;
- dfSales: The sales from last twelve months for each product;
- dfLastForecast: Previous sales forecast.

In [4]:
dfGroupAnalyst = pd.read_excel('./Inputs/0_Group per analyst.xls')
dfProd = pd.read_excel('./Inputs/1_Products.xls', sheet_name='Plan1', usecols='A:D,F,G,K')
dfStockEcmAndDc = pd.read_excel('./Inputs/1_STOCK ECM AND DC.xls', sheet_name='Plan1')
dfStockStores = pd.read_excel('./Inputs/1_STOCK STORES.xls', sheet_name='Plan1')
dfSales = pd.read_excel('./Inputs/2_SALES.xlsx', usecols='A, D, H, J:K')
dfLastForecast = pd.read_excel('./Inputs/2_LAST FORECAST.xlsx')

TransformColumns(dfProd)
TransformColumns(dfStockEcmAndDc)
TransformColumns(dfStockStores)
TransformColumns(dfSales)

From our Product Dataframe (dfProd), let's filter out all products that have been discontinued and that are unavailabe with no reorder, as well as all products where 'griffe' is different than Brand A. Due to business specifics, let's also filter out Groups E and AT.

In [5]:
dfProd = dfProd.loc[dfProd.linha != 'DISCONTINUED', :]
dfProd = dfProd.loc[dfProd.linha != 'UNAVAILABLE - NO REORDER', :]
dfProd = dfProd.loc[dfProd.griffe == 'Brand A', :]
dfProd = dfProd.loc[dfProd.grupo_produto != 'Group E', :]
dfProd = dfProd.loc[dfProd.grupo_produto != 'Group AT', :]

Now let's divide the stock DataFrame into two separate DataFrames: one for the e-commerce ad another for the physical stores.

In [6]:
dfStockEcmAndDc['estoque_cd'] = dfStockEcmAndDc['disponivel_venda'] + dfStockEcmAndDc['movimentação_cd']

dfECM = dfStockEcmAndDc.loc[dfStockEcmAndDc.filial == 'ECOMMERCE', :]
dfDC = dfStockEcmAndDc.loc[dfStockEcmAndDc.filial == 'DC', :]

### Handling the Dataframes

In order to slice the relevant period for the analysis, we will apply varQuery to dfSales. Then, the minimum month and year from the register date is going to be stored to variables.

In [7]:
dfSales = dfSales.query(varQuery)

mesmin = dfSales['data_registro'].min().month
anomin = dfSales['data_registro'].min().year

With only the correct period on the DataFrame, let's transform the columns 'mes' and 'ano' into one string representation of the month in question: 'mm' and 'yyyy' into 'mm / yyyy'. Pivot the sales.

In [8]:
dfSales['ano_e_mes'] = dfSales.apply(lambda x: str(dictMonths[x['mes']])+'/'+str(x['ano']), axis=1)
dfSales = dfSales.pivot_table(values='venda', index='produto', columns='ano_e_mes', aggfunc='sum')

In order to walk towards our final solution, we will start merging the DataFrames. Let's first merge the product DataFrame with the list of groups per analyst and then iterate through the last twelve months of sales, merging each column in the correct order.

In [9]:
dfNewForecast = dfProd.merge(dfGroupAnalyst, on='grupo_produto', how='left')

for m in range(0, 12):
    if mesmin + m <= 12:
        mes = mesmin + m
        ano = anomin
    else:
        mes = mesmin + m -12
        ano = anomin +1

    mes_ano = str(dictMonths[mes])+'/'+str(ano)
    dfNewForecast = dfNewForecast.merge(dfSales[[mes_ano]], left_on='produto', right_index=True, how='left')

Let's now add the stock on stores, stock DC and Stock ECM. Finally create the column Stock stores+DC, needed for the later work to be developed.

In [10]:
dfNewForecast = dfNewForecast.merge(dfStockStores[['produto', 'estoque']], on='produto', how='left')
dfNewForecast.rename(index=str, columns={"estoque": "rede"}, inplace=True)

dfNewForecast = dfNewForecast.merge(dfDC[['produto', 'estoque_cd']], on='produto', how='left')
dfNewForecast.rename(index=str, columns={"estoque_cd": "cd"}, inplace=True)

dfNewForecast = dfNewForecast.merge(dfECM[['produto', 'estoque_cd']], on='produto', how='left')
dfNewForecast.rename(index=str, columns={"estoque_cd": "ecm"}, inplace=True)

dfNewForecast = dfNewForecast.fillna(0)
dfNewForecast['estoque_rede+cd'] = dfNewForecast['rede'] + dfNewForecast['cd']

Let's then create a list of the 12 future months and populate the columns from the new Forecast with the sales previously predicted by the analysts.

In [11]:
mesesFuturos =[]

for m in range(0, 14):
    if mesmin + m <= 12:
        mes = mesmin + m
        ano = anomin + 1
    else:
        mes = mesmin + m -12
        ano = anomin + 2
    mes_ano = str(dictMonths[mes])+'/'+str(ano)
    mesesFuturos.append(mes_ano)


for m in mesesFuturos[:-1]:
    dfNewForecast[m] = dfNewForecast.apply(lambda x: dfLastForecast.loc[(dfLastForecast['produto'] == x['produto'])][m].sum(), 
                                           axis=1)
for m in mesesFuturos[-1:]:
    dfNewForecast[m] = dfNewForecast[dfNewForecast.columns.tolist()[-1]]

# Exporting the file

Save the new file to excel.

In [12]:
dfNewForecast.to_excel("NewForecast.xls", sheet_name='Base', index=False)